In [1]:
import copy
import os
import numpy as np
from numpy.linalg import norm
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin"

# Path for .pkl files
# target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\"
target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\mixture_shocktube\\pilot_runs\\"
# Path for Quail source code
source_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\"

In [2]:
# Import quail modules
os.chdir(source_dir)
import argparse
import importlib
import sys

import defaultparams as default_deck
import errors
from general import ShapeType, SolverType, PhysicsType

import meshing.common as mesh_common
import meshing.gmsh as mesh_gmsh
import meshing.tools as mesh_tools

import numerics.helpers.helpers as helpers
import numerics.timestepping.tools as stepper_tools

import physics.zerodimensional.zerodimensional as zerod
import physics.euler.euler as euler
import physics.navierstokes.navierstokes as navierstokes
import physics.scalar.scalar as scalar
import physics.chemistry.chemistry as chemistry
import physics.multiphasevpT.multiphasevpT as multiphasevpT

import processing.readwritedatafiles as readwritedatafiles
import processing.post as post
import processing.plot as plot
import processing

import solver.DG as DG
import solver.ADERDG as ADERDG
import solver.tools as solver_tools

import time
import multiprocessing as mp
from multidomain import Domain, Observer

os.chdir(target_dir)

In [ ]:
%%capture
%run "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\quail" "conduit.py"
# Run box to run

In [ ]:
# Waves first arrive at left boundary of [-600,600] at ~ output20 (400ms)
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_21.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_35.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit2_35.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_35.pkl")
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_52.pkl")
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_163.pkl")
#
#  for i in range(6):
#   plt.subplot(2,3,i+1)
#   plt.semilogy(solver.state_coeffs[:,:,i])

if solver.physics.NDIMS == 1:
	pltqtys = ["pDensityA", "pDensityWv", "pDensityM", "XMomentum",
		"Energy", "pDensityWt", "Pressure", "SoundSpeed", "Temperature",
		"XVelocity", "volFracM", "pDensityC"]
	equal_aspect_ratio = False
else:
	pltqtys = ["pDensityA", "XMomentum", "YMomentum", "Pressure", "Temperature"]
	equal_aspect_ratio = True

for pltqty in pltqtys:
	plot.plot_solution(solver.mesh, solver.physics, solver, pltqty,
				plot_numerical=True, plot_exact=False, plot_IC=False,
				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
				equidistant_pts=False, include_mesh=False, regular_2D=False,
				equal_AR=equal_aspect_ratio)

In [ ]:
idx = 163
# solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{idx}.pkl")
# plot.plot_solution(solver.mesh, solver.physics, solver, "Pressure",
# 				plot_numerical=True, plot_exact=False, plot_IC=False,
# 				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
# 				equidistant_pts=False, include_mesh=False, regular_2D=False,
# 				equal_AR=equal_aspect_ratio)
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm2_{idx}.pkl")
plot.plot_solution(solver.mesh, solver.physics, solver, "Pressure",
				plot_numerical=True, plot_exact=False, plot_IC=False,
				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
				equidistant_pts=False, include_mesh=False, regular_2D=False,
				equal_AR=equal_aspect_ratio)

In [ ]:
import ternary
%matplotlib tk

idx = 5
# solver_atm1 = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{idx}.pkl")
# solver_atm2 = readwritedatafiles.read_data_file(f"mixture_shocktube_atm2_{idx}.pkl")
# solver_conduit1 = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{idx}.pkl")
# solver_conduit2 = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit2_{idx}.pkl")
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{idx}.pkl")
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{idx}.pkl")
solver_conduit2 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit2_{idx}.pkl")

fontsize = 12

def prep_ternary():
  ## Boundary and Gridlines
  scale = 100.0
  figure, tax = ternary.figure(scale=scale)

  # Draw Boundary and Gridlines
  tax.boundary(linewidth=2.0)
  tax.gridlines(color="black", multiple=10)
  tax.gridlines(color="blue", multiple=5, linewidth=0.1)

  # Set Axis labels and Title
  # Set ticks
  tax.ticks(axis='lbr', multiple=10, linewidth=2)

  ternary.plt.show()
  plt.box(False)

  # Remove default Matplotlib Axes
  tax.clear_matplotlib_ticks()
  return tax

def get_wt_percents(solver, coordset="AWvM"):
  rho = np.sum(solver.state_coeffs[:,:,solver.physics.get_mass_slice()], axis=2)
  if coordset == "AWvM":
    y1 = solver.state_coeffs[:,:,0] / rho * 100.0
    y2 = solver.state_coeffs[:,:,1] / rho * 100.0
    y3 = solver.state_coeffs[:,:,2] / rho * 100.0
    return list(zip(y1.ravel(), y2.ravel(), y3.ravel()))
  elif coordset == "AWtMd":
    y1 = solver.state_coeffs[:,:,0] / rho * 100.0
    # Water transform
    arhoWt = solver.state_coeffs[:,:,solver.physics.get_state_index("pDensityWt")]
    arhoWd = arhoWt - solver.state_coeffs[:,:,1]
    y2 = arhoWt / rho * 100.0
    y3 = (solver.state_coeffs[:,:,2]-arhoWd) / rho * 100.0
  else:
    raise KeyError("Invalid coordset.")
  return list(zip(y1.ravel(), y2.ravel(), y3.ravel()))

''' Air, exsolved water, magma mixture'''
tax = prep_ternary()
tax.bottom_axis_label("wt% air", fontsize=fontsize)
tax.right_axis_label("wt% water, exsolved", fontsize=fontsize)
tax.left_axis_label("wt% frac melt + crystals + water, dissolved", fontsize=fontsize)

tax.right_corner_label("Air", fontsize=fontsize)
tax.top_corner_label("Exsolved water", fontsize=fontsize)
tax.left_corner_label("Magma", fontsize=fontsize)

tax.scatter(get_wt_percents(solver_atm1, "AWvM"), color='blue')
tax.scatter(get_wt_percents(solver_atm2, "AWvM"), color='red')
tax.scatter(get_wt_percents(solver_conduit1, "AWvM"), color='magenta')
tax.scatter(get_wt_percents(solver_conduit2, "AWvM"), color='cyan')
# tax.set_title("", fontsize=fontsize)

tax = prep_ternary()
tax.bottom_axis_label("wt% air", fontsize=fontsize)
tax.right_axis_label("wt% water, total", fontsize=fontsize)
tax.left_axis_label("wt% frac melt + crystals", fontsize=fontsize)

tax.right_corner_label("Air", fontsize=fontsize)
tax.top_corner_label("Total water", fontsize=fontsize)
tax.left_corner_label("Dry magma", fontsize=fontsize)

tax.scatter(get_wt_percents(solver_atm1, "AWtMd"), marker='d', color='blue', label="atm near")
tax.scatter(get_wt_percents(solver_atm2, "AWtMd"), marker='d', color='brown', label="atm far")
tax.scatter(get_wt_percents(solver_conduit1, "AWtMd"), marker='s', color='magenta', label="conduit upper")
tax.scatter(get_wt_percents(solver_conduit2, "AWtMd"), marker='s',color='cyan', label="conduit lower")

tax.legend()

In [ ]:
%matplotlib tk
idx = 120
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{idx}.pkl")
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{idx}.pkl")
solver_conduit2 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit2_{idx}.pkl")

from matplotlib.animation import FuncAnimation
fig = plt.figure()
# ax = plt.axes(xlim=(-2400, 0), ylim=(0, 100e6)) # p
ax = plt.axes(xlim=(-2400, 0), ylim=(-300, 500)) # u, c
# ax = plt.axes(xlim=(-2400, 0), ylim=(-1, 10e9)) # wv
# ax = plt.axes()
line, = ax.plot([], [], lw=2)

def get_nodes(solver):
  nodal_pts = solver.basis.get_nodes(solver.order)
  # Allocate [ne] x [nb, ndims]
  x = np.empty((solver.mesh.num_elems,) + nodal_pts.shape)
  for elem_ID in range(solver.mesh.num_elems):
    # Fill coordinates in physical space
    x[elem_ID] = mesh_tools.ref_to_phys(solver.mesh, elem_ID, nodal_pts)
  return x

def update(frame_index):
  frame_skip = 200
  stride = 10
  solver_conduit1 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_conduit1_{stride*frame_index+frame_skip}.pkl")
  x1 = get_nodes(solver_conduit1).ravel()
  p1 = solver_conduit1.physics.compute_additional_variable("Pressure", 
    solver_conduit1.state_coeffs, False).ravel()
  a1 = solver_conduit1.physics.compute_additional_variable("SoundSpeed", 
    solver_conduit1.state_coeffs, False).ravel()
  u1 = solver_conduit1.physics.compute_additional_variable("XVelocity", 
    solver_conduit1.state_coeffs, False).ravel()
  q1 = a1

  solver_conduit2 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_conduit2_{stride*frame_index+frame_skip}.pkl")
  x2 = get_nodes(solver_conduit2).ravel()
  p2 = solver_conduit2.physics.compute_additional_variable("Pressure", 
    solver_conduit2.state_coeffs, False).ravel()
  a2 = solver_conduit2.physics.compute_additional_variable("SoundSpeed", 
    solver_conduit2.state_coeffs, False).ravel()
  u2 = solver_conduit2.physics.compute_additional_variable("XVelocity", 
    solver_conduit2.state_coeffs, False).ravel()
  q2 = a2

  x = np.concatenate((x2, x1))
  p = np.concatenate((q2, q1))
  
  line.set_data(x, p)
  return line,

# plot.plot_solution(solver_conduit1.mesh, solver_conduit1.physics, solver_conduit1, "Pressure",
#   plot_numerical=True, plot_exact=False, plot_IC=False,
#   create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
#   equidistant_pts=False, include_mesh=False, regular_2D=False,
#   equal_AR=equal_aspect_ratio)
# plot.plot_solution(solver_conduit2.mesh, solver_conduit2.physics, solver_conduit2, "Pressure",
#   plot_numerical=True, plot_exact=False, plot_IC=False,
#   create_new_figure=False, ylabel=None, fmt='b.', legend_label="DG",
#   equidistant_pts=False, include_mesh=False, regular_2D=False,
#   equal_AR=equal_aspect_ratio)
# solver_conduit1.time

def init():
    line.set_data([], [])
    return line,

anim = FuncAnimation(fig, update, frames=100,
    init_func=init, blit=True)
  
# anim.save('basic_animation_test', fps=30)

plt.show()




In [18]:
%matplotlib tk
import processing.mdtools as mdt
solver_atm1_init = readwritedatafiles.read_data_file(f"v0_phi0600m_atm1_{0}.pkl")
solver_conduit1_init = readwritedatafiles.read_data_file(f"v0_phi0600m_conduit1_{0}.pkl")
mdt.viz(solver_atm1_init, "Pressure")
# solver_atm1_init.physics.compute_additional_variable("Pressure", solver_atm1_init.state_coeffs, False).max()
# solver_conduit1_init.physics.compute_additional_variable("Pressure", solver_conduit1_init.state_coeffs[-1:,:,:], False).min()

In [ ]:
equal_aspect_ratio = True
plot_idx = 1200
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{plot_idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{plot_idx}.pkl")
plot.plot_solution(solver_atm1.mesh, solver_atm1.physics, solver_atm1, "Velocity",
  plot_numerical=True, plot_exact=False, plot_IC=False,
  create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
  equidistant_pts=False, include_mesh=False, regular_2D=False,
  equal_AR=equal_aspect_ratio)
plot.plot_solution(solver_atm1.mesh, solver_atm1.physics, solver_atm2, "Velocity",
  plot_numerical=True, plot_exact=False, plot_IC=False,
  create_new_figure=False, ylabel=None, fmt='b.', legend_label="DG",
  equidistant_pts=False, include_mesh=False, regular_2D=False,
  equal_AR=equal_aspect_ratio)

In [ ]:
# cb = plt.colorbar()
# ax2 = cb.ax.twinx()
# ax2.set_ylim(clims)
plt.clf()
# sm = plt.cm.ScalarMappable(norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]), cmap=plt.get_cmap())

In [ ]:
def downsample(solver, plot_qty:str="Pressure"):
	solver = copy.deepcopy(solver)
	# Downsample
	original_order = solver.order
	solver.order = 0
	equidistant_pts = True
	x = plot.get_sample_points(solver.mesh, solver, solver.physics, solver.basis,
				equidistant_pts)
	solver.order = original_order
	# Compute on downsampled points
	var_plot = plot.get_numerical_solution(solver.physics, solver.state_coeffs, x,
					solver.basis, plot_qty)
	return x, var_plot

def custom_plot_2D(x, var_plot, solver, levels=None):
	# plot.plot_2D_general(solver.physics, x, var_plot)

	if levels is None:
		''' Open temp figure to get default levels'''
		figtmp = plt.figure()
		# Run this for the sole purpose of getting default contour levels
		levels = plot.plot_2D_regular(solver.physics, np.copy(x), np.copy(var_plot))
		plt.close(figtmp)

	num_elems = x.shape[0]
	for elem_ID in range(num_elems):
		# Triangulate each element one-by-one
		tris, utri = plot.triangulate(solver.physics, x[elem_ID], var_plot[elem_ID])
		# Plot
		plt.tricontourf(tris, utri, levels=levels, extend="both")
		if np.mod(elem_ID,100) == 0:
			plt.pause(0.05)

	# Adjust plot
	plt.gca().set_aspect('equal', adjustable='box')
	plt.xlabel("$x$")
	plt.ylabel("$y$")
	return var_plot

def plot_mean(x, q, clims):
	''' Create very cheap plot '''
	cmap = plt.get_cmap()
	cnorm = matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]) 
	for i in range(x.shape[0]):
		pp = plt.Polygon([[x[i,0,0], x[i,0,1]],
			[x[i,1,0], x[i,1,1]],
			[x[i,2,0], x[i,2,1]]], facecolor=cmap(cnorm(q[i,:,0].mean())))
		plt.gca().add_patch(pp)
	plt.axis("auto")
	plt.axis("equal")

def plot_mean1D(x, q, clims, xscale=1.0, xshift=0.0):
	''' Create very cheap plot '''
	cmap = plt.get_cmap()
	cnorm = matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]) 
	for i in range(x.shape[0]):
		pp = plt.Rectangle((xshift, x[i,0,0]), width=xscale, height=x[i,1,0]-x[i,0,0],
			facecolor=cmap(cnorm(q[i,:,0].mean())))
		plt.gca().add_patch(pp)
	plt.axis("auto")
	plt.axis("equal")

plot_idx = 1200
solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm1_{plot_idx}.pkl")
solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0800m_atm2_{plot_idx}.pkl")

x1, p1 = downsample(solver_atm1, "Pressure")
x2, p2 = downsample(solver_atm2, "Pressure")
clims = np.min([p1.min(), p2.min()]), np.max([p1.max(), p2.max()])
levels = np.linspace(clims[0], clims[1], 10)
# custom_plot_2D(x1, p1, solver_atm1, levels=levels)
# custom_plot_2D(x2, p2, solver_atm2, levels=levels)
plot_mean(x1, p1, clims)
plot_mean(x2, p2, clims)

sm = plt.cm.ScalarMappable(
	norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
	cmap=plt.get_cmap())
cb = plt.colorbar(sm)
cb.set_label("p (Pa)")


In [92]:
%matplotlib tk
from processing.mdtools import generate_anim

generate_anim(["v0_phi0500m_atm1", "v0_phi0500m_atm2"],
  ["v0_phi0500m_conduit1"],
  "output_p_0500",
  12,
  "Pressure",
  filter=lambda x, x0: np.log10(np.abs(x-x0)+1e-7),
  initial=1,
  stride=5,
  is_high_detail=False)

Animation constructed


In [ ]:
solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0800m_conduit1_{plot_idx}.pkl")

xc1, pc1 = downsample(solver_conduit1, "Pressure")
plot_mean1D(xc1, pc1, clims, xscale=50, xshift=0.0)
plt.axis("auto")
plt.axis("equal")

In [91]:
solver_atm1_init.bface_helpers.quad_wts
# solver_atm1_init.bface_helpers.x_bgroups[bgroup.number]
bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
solver_atm1_init.bface_helpers.x_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]
quad_wts = solver_atm1_init.bface_helpers.quad_wts # [nq_bdry, 1]
normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number] # [ne_bdry, nq_bdry, ndims]

# solver_atm1_init.
Uq_synthetic = np.ones_like(solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number])[:,:,0:1] # [ne_bdry, nq_bdry, ns]
# weighted_Uq = np.einsum('ijk, jm -> ijk', Uq_synthetic, quad_wts)

bgroup = solver_atm1_init.mesh.boundary_groups["x2"]
normals = solver_atm1_init.bface_helpers.normals_bgroups[bgroup.number]
# Compute boundary measure
boundary_length = solver_atm1_init.bface_helpers.face_lengths_bgroups[bgroup.number].sum()
# Compute average
U_avg = np.einsum('ijm, ijk, jn -> k',
  np.linalg.norm(normals, axis=2, keepdims=True),
  Uq_synthetic,
  solver_atm1_init.bface_helpers.quad_wts) / boundary_length

# U_avg = U_avg/boundary_length
U_avg

array([[[8.333333333274119],
        [8.333333333274119],
        [8.333333333274119]],

       [[8.333333333333336],
        [8.333333333333336],
        [8.333333333333336]],

       [[8.333333333392545],
        [8.333333333392545],
        [8.333333333392545]]])

In [ ]:
%matplotlib tk
initial = 0
stride = 10

fig = plt.figure()
# import celluloid
# import ffmpeg
# camera = celluloid.Camera(fig)
# plt.rcParams['animation.ffmpeg_path'] = 'C:\Users\User\Desktop'
plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin\\ffmpeg.exe"

# import ffmpeg
import matplotlib.animation as animation
FFwriter = animation.FFMpegWriter()
FFwriter.setup(plt.gcf(), "new_out_test_lol.mp4")

solver_atm1_init = readwritedatafiles.read_data_file(f"v0_phi0700m_atm1_{0}.pkl")
solver_atm2_init = readwritedatafiles.read_data_file(f"v0_phi0700m_atm2_{0}.pkl")
solver_conduit1_init = readwritedatafiles.read_data_file(f"v0_phi0700m_conduit1_{0}.pkl")

for i in range(120):
	plt.clf()
	read_index = initial+i*stride
	solver_atm1 = readwritedatafiles.read_data_file(f"v0_phi0700m_atm1_{read_index}.pkl")
	solver_atm2 = readwritedatafiles.read_data_file(f"v0_phi0700m_atm2_{read_index}.pkl")

	if False:
		''' Mass flux magnitude '''
		x1, p1 = downsample(solver_atm1, "XMomentum")
		x1, q1 = downsample(solver_atm1, "YMomentum")
		p1 = np.sqrt(p1**2+q1**2)
		x2, p2 = downsample(solver_atm2, "XMomentum")
		x2, q2 = downsample(solver_atm2, "YMomentum")
		p2 = np.sqrt(p2**2+q2**2)
	x1, p1 = downsample(solver_atm1, "Pressure")
	x2, p2 = downsample(solver_atm2, "Pressure")

	_, p1_init = downsample(solver_atm1_init, "Pressure")
	_, p2_init = downsample(solver_atm2_init, "Pressure")

	p1 -= p1_init
	p2 -= p2_init

	filter = lambda z : np.log10(np.abs(z)+1e-10)
	p1 = filter(p1)
	p2 = filter(p2)

	''' Conduit '''
	solver_conduit1 = readwritedatafiles.read_data_file(f"v0_phi0700m_conduit1_{read_index}.pkl")
	xc1, pc1 = downsample(solver_conduit1, "Pressure")
	_, pc1_init = downsample(solver_conduit1_init, "Pressure")
	pc1 -= pc1_init
	pc1 = filter(pc1)

	# Auto clim
	clims = np.min([p1.min(), p2.min(), pc1.min()]), np.max([p1.max(), p2.max(), pc1.max()])

	clims = ([0,8])
	plot_mean(x1, p1, clims)
	plot_mean(x2, p2, clims)	
	
	plot_mean1D(xc1, pc1, clims, xscale=50, xshift=0.0)

	sm = plt.cm.ScalarMappable(
		norm=matplotlib.colors.Normalize(vmin=clims[0], vmax=clims[1]),
		cmap=plt.get_cmap())
	cb = plt.colorbar(sm)
	# cb.set_label("p (Pa)")
	
	plt.axis("auto")
	plt.axis("equal")
	plt.pause(0.010)
	# Grab frame with FFwriter
	FFwriter.grab_frame()

print("Done")
FFwriter.finish()

In [ ]:
%matplotlib tk

from matplotlib.animation import FuncAnimation
fig = plt.figure()
# ax = plt.axes(xlim=(-2400, 0), ylim=(0, 100e6)) # p
ax = plt.axes(xlim=(-2400, 0), ylim=(-300, 500)) # u, c
# ax = plt.axes(xlim=(-2400, 0), ylim=(-1, 10e9)) # wv
# ax = plt.axes()
line, = ax.plot([], [], lw=2)

def get_nodes(solver):
  nodal_pts = solver.basis.get_nodes(solver.order)
  # Allocate [ne] x [nb, ndims]
  x = np.empty((solver.mesh.num_elems,) + nodal_pts.shape)
  for elem_ID in range(solver.mesh.num_elems):
    # Fill coordinates in physical space
    x[elem_ID] = mesh_tools.ref_to_phys(solver.mesh, elem_ID, nodal_pts)
  return x

def update(frame_index):
  frame_skip = 200
  stride = 10
  solver_atm1 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_atm1_{stride*frame_index+frame_skip}.pkl")
  x1 = get_nodes(solver_atm1).ravel()
  p1 = solver_atm1.physics.compute_additional_variable("Pressure", 
    solver_atm1.state_coeffs, False).ravel()
  a1 = solver_atm1.physics.compute_additional_variable("SoundSpeed", 
    solver_atm1.state_coeffs, False).ravel()
  u1 = solver_atm1.physics.compute_additional_variable("XVelocity", 
    solver_atm1.state_coeffs, False).ravel()
  q1 = a1

  solver_atm2 = readwritedatafiles.read_data_file(
    f"v0_phi0800m_atm2_{stride*frame_index+frame_skip}.pkl")
  x2 = get_nodes(solver_atm2).ravel()
  p2 = solver_atm2.physics.compute_additional_variable("Pressure", 
    solver_atm2.state_coeffs, False).ravel()
  a2 = solver_atm2.physics.compute_additional_variable("SoundSpeed", 
    solver_atm2.state_coeffs, False).ravel()
  u2 = solver_atm2.physics.compute_additional_variable("XVelocity", 
    solver_atm2.state_coeffs, False).ravel()
  q2 = a2

  x = np.concatenate((x2, x1))
  p = np.concatenate((q2, q1))
  plot_mean(x, p, clims)
  
  line.set_data(x, p)
  return line,

def init():
    line.set_data([], [])
    return line,

anim = FuncAnimation(fig, update, frames=100,
    init_func=init, blit=True)
  
# anim.save('basic_animation_test', fps=30)

plt.show()

In [ ]:
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_288.pkl")

if solver.physics.NDIMS == 1:
	pltqtys = ["pDensityA", "pDensityWv", "pDensityM", "XMomentum",
		"Energy", "pDensityWt", "Pressure", "SoundSpeed", "Temperature",
		"XVelocity", "volFracM", "pDensityC"]
	equal_aspect_ratio = False
else:
	pltqtys = ["pDensityA", "XMomentum", "YMomentum", "Pressure", "Temperature"]
	equal_aspect_ratio = True

for pltqty in pltqtys:
	plot.plot_solution(solver.mesh, solver.physics, solver, pltqty,
				plot_numerical=True, plot_exact=False, plot_IC=False,
				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
				equidistant_pts=False, include_mesh=False, regular_2D=False,
				equal_AR=equal_aspect_ratio)

In [ ]:
# Minimum of crystal partial density
np.min(solver.state_coeffs[:,:,6])

solver.state_coeffs[:,:,4]

In [ ]:
snapshotidx = 163
snapshotidx = 163

solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{snapshotidx}.pkl")
x_conduit = solver.mesh.node_coords
p_conduit = solver.physics.compute_additional_variable("Pressure", solver.state_coeffs, False)

solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{snapshotidx}.pkl")

p = solver.physics.compute_additional_variable("Pressure", solver.state_coeffs, False)
mesh = solver.mesh
# plt.plot(p.ravel())
# plt.scatter(mesh.node_coords[:,0], mesh.node_coords[:,:,1], p)
mesh.node_coords.shape
x = plot.get_sample_points(
			mesh, solver, solver.physics, solver.basis, True)
plt.scatter(x.mean(axis=1)[:,0], x.mean(axis=1)[:,1],
  c=p.mean(axis=1)[:,0])
# x.mean(axis=1).shape
# p.mean(axis=1)[:,0].shape
plt.axis("equal")

plt.figure()
# plt.plot(x.mean(axis=1)[:,1], p.mean(axis=1)[:,0], '.')
plt.plot(x.mean(axis=1)[:,1]+150.0, p[:,:,0], '.')
# p[:,:,0].shape

# Plot conduit
xnodes_conduit = np.zeros_like(p_conduit)
xnodes_conduit[:,0,:] = x_conduit[:-1]
xnodes_conduit[:,2,:] = x_conduit[1:]
xnodes_conduit[:,1,:] = 0.5*xnodes_conduit.sum(axis=1)
# plt.figure()
plt.plot(xnodes_conduit.ravel()-600, p_conduit.ravel() ,'.-')
plt.xlim([-100,100])
plt.ylim([5e4,300000])

In [ ]:
snapshotidx = 82
solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{snapshotidx}.pkl")
num_saves = 82# 96
size_tuple = list(solver.state_coeffs[:,:,0:1].shape)
size_tuple.append(num_saves+1)
p_hist = np.zeros(tuple(size_tuple))
save_count = range(num_saves+1)
# x for 2D domain
x = plot.get_sample_points(
			solver.mesh, solver, solver.physics, solver.basis, True)

solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{snapshotidx}.pkl")
size_tuple = list(solver.state_coeffs[:,:,0:1].shape)
size_tuple.append(num_saves+1)
x_conduit = plot.get_sample_points(
			solver.mesh, solver, solver.physics, solver.basis, True)
p_conduit_hist = np.zeros(tuple(size_tuple))


key_quant = "Pressure"
for i in save_count:
  solver = readwritedatafiles.read_data_file(f"mixture_shocktube_conduit_{i}.pkl")
  p_conduit_hist[:,:,:,i] = solver.physics.compute_additional_variable(key_quant, solver.state_coeffs, False)
  # p_conduit_hist[:,:,:,i] = solver.state_coeffs[:,:,2:3]
  solver = readwritedatafiles.read_data_file(f"mixture_shocktube_atm1_{i}.pkl")
  p_hist[:,:,:,i] = solver.physics.compute_additional_variable(key_quant, solver.state_coeffs, False)
  # p_hist[:,:,:,i] = solver.state_coeffs[:,:,2:3]
  # Approximate energy
  # E_2D = np.sum(solver.state_coeffs[:,:,5])
  


# Problematic location
# plot.plot_solution(solver.mesh, solver.physics, solver, "Pressure",
# 				plot_numerical=True, plot_exact=False, plot_IC=False,
# 				create_new_figure=True, ylabel=None, fmt='b.', legend_label="DG",
# 				equidistant_pts=False, include_mesh=False, regular_2D=False,
# 				equal_AR=equal_aspect_ratio)
# plt.ylim([-150, -100])
# plt.xlim([0,50])
# plt.clim([-30000,60000*0])

# plt.figure()
# plt.plot(x.mean(axis=1)[:,1], p.mean(axis=1)[:,0], '.')
# plt.plot(x.mean(axis=1)[:,1]+150.0, p[:,:,0], '.')
# p[:,:,0].shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib tk
# p = solver.physics.compute_additional_variable("Pressure", solver.state_coeffs, False)
# p [258,:,:]
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

from matplotlib import cm

pseudo_x = x.mean(axis=1)[:,1]+150.0
ordering = np.argsort(pseudo_x)
X, Y = np.meshgrid(pseudo_x[ordering], save_count)
Z = p_hist[ordering,0,0,:].squeeze().T

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
# p_hist[:,:,-1]
# plt.xlabel("x")

ordering = np.argsort(pseudo_x)

plt.figure()
for i in (save_count):
  plt.plot(pseudo_x[ordering], p_hist[ordering,0,0,i], '.-')
  # plt.hold(True)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

x_conduit.mean(axis=1).shape
pseudo_x_conduit = x_conduit.mean(axis=1)[:,0]-200
p_conduit_hist.mean(axis=1)[:,0,:]

ordering = np.argsort(pseudo_x_conduit)
X, Y = np.meshgrid(pseudo_x_conduit[ordering], save_count)
Z = p_conduit_hist.mean(axis=1)[ordering,0,:].squeeze().T

# # Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)



In [ ]:
x_global = np.append(pseudo_x, pseudo_x_conduit)
p_global = np.append(p_hist.squeeze(), p_conduit_hist.mean(axis=1)[:,0,:], axis=0)
# p_hist.squeeze().shape
# p_conduit_hist.mean(axis=1)[:,0,:].shape

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

# x_global.mean(axis=1).shape

ordering = np.argsort(x_global)
X, Y = np.meshgrid(x_global[ordering], save_count)
Z = p_global[ordering,:].squeeze().T

# # Plot the surface.
surf = ax.plot_surface(X, Y, np.log10(Z), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)




In [ ]:
assert(solver.physics.NDIMS == 2)
# np.argmin(p_hist[117,:,:])
p_hist[117,:,:,-1]
solver.state_coeffs[117,:,:]
# solver.physics.compute_additional_variable("Temperature", solver.state_coeffs, False)[258,:,:]